In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical

In [2]:
# Carregando dados
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [3]:
# Normalização dos elementos
x_train = x_train.reshape((60000, 28, 28, 1)).astype('float32') / 255
x_test = x_test.reshape((10000, 28, 28, 1)).astype('float32') / 255

In [4]:
x_train.shape #60000 imagens, 28x28px com 1 camada

(60000, 28, 28, 1)

In [5]:
#One hot enconding das labels ->>>>>> Pesquisar
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [10]:
# Definindo o bloco residual
def residual_block(x, filters, kernel_size=3, stride=1):
    # Saida = Ativação(Normalização(Convolução(Entrada)))
    y = layers.Conv2D(filters, kernel_size=kernel_size, strides=stride, padding='same')(x)
    y = layers.BatchNormalization()(y)
    y = layers.Activation('relu')(y)

    y = layers.Conv2D(filters, kernel_size=kernel_size, padding='same')(y)
    y = layers.BatchNormalization()(y)

    if stride > 1: #Garantir que as dimensões sejam iguais
        x = layers.Conv2D(filters, kernel_size=1, strides=stride, padding='same')(x)
    
    out = layers.add([x, y])
    out = layers.Activation('relu')(out)
    return out

In [11]:
# Construindo a arquitetura ResNet para MNIST
input_shape = (28, 28, 1)
num_classes = 10

inputs = tf.keras.Input(shape=input_shape)
x = layers.Conv2D(64, 7, strides=2, padding='same')(inputs) #Se same e stride 1, tamanho "convolucionado" = tamanho imagens inicial Retorna tensor 4d
x = layers.BatchNormalization()(x)
x = layers.Activation('relu')(x)
x = layers.MaxPooling2D(3, strides=2, padding='same')(x)

In [12]:
# Adicionando as conexões entre os blocos residuais
for _ in range(3): # Posso aumentar profundidade da minha rede
    x = residual_block(x, 64) #64 Filtros

x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(num_classes, activation='softmax')(x)

In [6]:
# Criando o modelo
model = models.Model(inputs, outputs)

NameError: name 'inputs' is not defined

In [14]:
# Compilando o modelo
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [16]:
# Treinando o modelo
model.fit(x_train, y_train, epochs=5, batch_size=64, validation_split=0.2)

Epoch 1/5
750/750 [==============================] - 73s 95ms/step - loss: 0.1297 - accuracy: 0.9617 - val_loss: 0.0858 - val_accuracy: 0.9734
Epoch 2/5
750/750 [==============================] - 66s 88ms/step - loss: 0.0497 - accuracy: 0.9845 - val_loss: 0.0641 - val_accuracy: 0.9803
Epoch 3/5
750/750 [==============================] - 64s 85ms/step - loss: 0.0367 - accuracy: 0.9887 - val_loss: 0.0501 - val_accuracy: 0.9842
Epoch 4/5
750/750 [==============================] - 62s 83ms/step - loss: 0.0295 - accuracy: 0.9909 - val_loss: 0.1103 - val_accuracy: 0.9676
Epoch 5/5
750/750 [==============================] - 63s 84ms/step - loss: 0.0269 - accuracy: 0.9912 - val_loss: 0.0418 - val_accuracy: 0.9877


In [17]:
# Avaliando
test_loss, test_acc = model.evaluate(x_test, y_test)
print(f'Test accuracy: {test_acc}')

313/313 [==============================] - 3s 10ms/step - loss: 0.0476 - accuracy: 0.9837
Test accuracy: 0.9836999773979187
